참고  
* https://www.maartengrootendorst.com/blog/bertopic/  
* https://hackernoon.com/nlp-tutorial-topic-modeling-in-python-with-bertopic-372w35l9

In [ ]:
!pip install bertopic[visualization]

     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 5.2 MB 54.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.5 MB 37.6 MB/s 
     |████████████████████████████████| 1.2 MB 54.2 MB/s 
     |████████████████████████████████| 67 kB 6.0 MB/s 
     |████████████████████████████████| 636 kB 65.2 MB/s 
     |████████████████████████████████| 6.8 MB 37.2 MB/s 
     |████████████████████████████████| 895 kB 65.5 MB/s 
     |████████████████████████████████| 1.1 MB 49.4 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330746 sha256=c3feb508ec2e08584787e589192feca72660177969723d5c75028934a8b4e92e
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef

SBERT를 이용한 토픽 모델인 BERTopic은 별도 논문은 나오지 않은 모델이지만, 개발자는 BERTopic이 LDA를 대체할 수 있을만큼의 기술이라고 확신한다고 합니다. 여기서는 BERT 기반의 토픽 모델링 구현체인 BERTopic의 간단한 사용 방법에 대해서 다룹니다.

# 1. BERTopic

BERTopic은 BERT embeddings과 클래스 기반(class-based) TF-IDF를 활용하여 주제 설명에서 중요한 단어를 유지하면서도 쉽게 해석할 수 있는 조밀한 클러스터를 만드는 토픽 모델링 기술입니다. BERTopic 알고리즘은 크게 세 가지 과정을 거칩니다.

**1) 텍스트 데이터를 SBERT로 임베딩합니다.**  
알고리즘은 SBERT를 사용하여 문서를 임베딩합니다. 이때, BERTopic은 기본적으로 아래의 BERT들을 사용합니다.  

* **"paraphrase-MiniLM-L6-v2"** : 영어 데이터로 학습된 SBERT 
* **"paraphrase-multilingual-MiniLM-L12-v2"** : 50개 이상의 언어로 학습된 다국어 SBERT

**2) 문서를 군집화합니다.**  
UMAP을 사용하여 임베딩의 차원을 줄이고 HDBSCAN 기술을 사용하여 차원 축소된 임베딩을 클러스터링하고 의미적으로 유사한 문서 클러스터를 생성합니다.

**3) 토픽 표현을 생성**  
마지막 단계는 클래스 기반 TF-IDF로 토픽을 추출합니다.

# 2. 데이터 로드

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

In [ ]:
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

In [ ]:
# 상위 5개의 샘플 출력
docs[:5]

["\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n",
 'My brother is in the market for a high-performance video card that supports\nVESA local bus with 1-2MB RAM.  Does anyone have suggestions/ideas on:\n\n  - Diamond Stealth Pro Local Bus\n\n  - Orchid Farenheit 1280\n\n  - ATI Graphics Ultra Pro\n\n  - Any other high-per

In [ ]:
print('총 문서의 수 :', len(docs))

총 문서의 수 : 18846


# 3. 토픽 모델링

BERTopic의 fit_transform 메소드에 문자열들의 리스트를 입력으로 넣으면 토픽 모델링을 수행합니다.

In [ ]:
model = BERTopic()
topics, probabilities = model.fit_transform(docs)

In [ ]:
print('각 문서의 토픽 번호 리스트 :',len(topics))
print('첫번째 문서의 토픽 번호 :', topics[0])

각 문서의 토픽 번호 리스트 : 18846
첫번째 문서의 토픽 번호 : 0


get_topic_info() 메소드를 사용하여 토픽의 개수, 토픽의 크기, 각 토픽에 할당된 단어들을 일부 볼 수 있습니다.

In [ ]:
# 각 토픽의 크기와 각 토픽에 할당된 단어들
model.get_topic_info()

,Topic,Count,Name
0,-1,6514,-1_to_the_of_is
1,0,1825,0_game_team_games_he
2,1,572,1_key_clipper_chip_encryption
3,2,525,2_ites_cheek_yep_huh
4,3,465,3_israel_israeli_jews_arab
...,...,...,...
207,206,10,206_fat_weight_insulin_muscle
208,207,10,207_beeps_error_machine_chimes
209,208,10,208_colormap_colormaps_color_readonly
210,209,10,209_eisa_isa_bus_motherboard


Count 열의 값을 모두 합하면 총 문서의 수입니다.

In [ ]:
model.get_topic_info()['Count'].sum()

18846

위의 출력에서 Topic -1이 가장 큰 것으로 보입니다. -1은 토픽이 할당되지 않은 모든 이상치 문서(outliers)들을 나타냅니다. 현재 0번 토픽부터 210번 토픽까지 있는데, 임의로 5번 토픽에 대해서 단어들을 출력해봅시다.  

get_topic() 메소드의 입력으로 보고자하는 토픽의 번호를 넣어줍니다.

In [ ]:
model.get_topic(5)

[('drive', 0.036501379524217024),
 ('scsi', 0.027358077330910547),
 ('drives', 0.0229861502896249),
 ('ide', 0.019274207233754368),
 ('disk', 0.01808211458113983),
 ('controller', 0.016803056719952875),
 ('hard', 0.013004806725656367),
 ('scsi2', 0.012107882273732159),
 ('bios', 0.009949766797753059),
 ('scsi1', 0.009350150086818809)]

# 4. 토픽 시각화

BerTopic을 사용하면 LDAvis와 매우 유사한 방식으로 생성된 토픽을 시각화할 수 있습니다. 시각화를 통해 생성된 토픽에 대해 더 많은 통찰력을 얻을 수 있습니다. 우선 visualize_topics() 메소드로 시각화를 진행해봅시다.

In [ ]:
model.visualize_topics()

# 5. 단어 시각화

Visualization_barchart() 메소드는 c-TF-IDF 점수에서 막대 차트를 만들어 각 토픽에 대해 선택된 단어들을 표시합니다. 각 토픽에 대해서 선택된 단어들을 비교할 수 있습니다.

In [ ]:
model.visualize_barchart()

# 6. 토픽 유사도 시각화

각 토픽들이 서로 얼마나 유사한지 시각화할 수도 있습니다. visualize_heatmap() 메소드를 사용하여 히트맵을 시각화합니다.  
해당 히트맵의 원하는 위치에 마우스를 갖다대면 실질적인 유사도 값을 확인할 수 있습니다.

In [ ]:
model.visualize_heatmap()

# 7. 토픽의 수 정하기

때때로 너무 많은 토픽이 생성되거나 너무 적은 토픽이 생성될 수 있습니다. 토픽의 수를 직접 정하고 싶다면 몇 가지 방법이 존재합니다.

첫번째 방법은 모델 객체 생성 시에 nr_topics 값으로 원하는 토픽 수를 입력하여 원하는 토픽의 수를 설정할 수 있습니다. BerTopic은 유사한 토픽들을 찾아 하나의 토픽으로 병합합니다.

In [ ]:
model = BERTopic(nr_topics=20)
topics, probabilities = model.fit_transform(docs)

In [ ]:
model.visualize_topics()

또 다른 방법은 모델이 자동으로 토픽의 수를 줄이도록 설정하는 것입니다. 이 옵션을 사용하려면 모델 객체 생성 시에 "nr_topics"의 값을 "auto"로 설정하면 됩니다.

In [ ]:
model = BERTopic(nr_topics="auto")
topics, probabilities = model.fit_transform(docs)

In [ ]:
# 각 토픽의 크기와 각 토픽에 할당된 단어들
model.get_topic_info()

,Topic,Count,Name
0,-1,6583,-1_to_the_of_and
1,0,1841,0_game_team_he_games
2,1,1074,1_image_jpeg_software_for
3,2,663,2_key_encryption_clipper_chip
4,3,658,3_god_that_he_of
...,...,...,...
140,139,10,139_skin_acne_dry_pimples
141,140,10,140_bits_48bit_bit_color
142,141,10,141_circumcision_medical_parents_foreskin
143,142,10,142_blacks_african_white_culture


토픽의 개수를 지정하지 않았을 때는 0번 토픽부터 210번 토픽까지 총 211개의 토픽이 존재하였으나, 자동으로 토픽의 수가 줄어들도록 설정하자 토픽의 수가 0번부터 143번까지 총 144개로 줄어든 것을 확인할 수 있습니다.

# 8. 임의의 문서에 대한 예측

In [ ]:
new_doc = docs[0]
print(new_doc)



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [ ]:
topics, probs = model.transform([new_doc])

In [ ]:
print('예측한 토픽 번호 :', topics)

예측한 토픽 번호 : [0]


# 9. 모델 저장과 로드

In [ ]:
model.save("my_topics_model")
BerTopic_model = BERTopic.load("my_topics_model")

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.

